# データサイエンス(Python)勉強会用 2022/01/21 (金)

### CIFAR-10データセット
<概要><br>
10種類の画像が60000枚(トレーニングデータ50000枚、テストデータ10000枚)入ったデータセット。<br>
(縦[32] x 横[32] x 色[3: RGB])<br>

◆種類<br>
0: 飛行機<br>
1: 自動車<br>
2: 鳥<br>
3: ネコ<br>
4: 鹿<br>
5: イヌ<br>
6: カエル<br>
7: 馬<br>
8: 船<br>
9: トラック

### 使用するモジュールを読み込み

In [ ]:
import keras
from keras.datasets import cifar10
from keras.models import Sequential
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.layers.core import Dense, Dropout, Activation, Flatten
import numpy as np
import matplotlib.pyplot as plt

from tensorflow.keras.utils import to_categorical
from tensorflow.keras.utils import plot_model

### CIFAR-10データセットを読み込み
各種データの中身を出力して確認する。

In [ ]:
(X_train, Y_train), (X_test, Y_test) = cifar10.load_data()

# X_Train: 学習用画像データ (枚数[50,000] x 縦[32] x 横[32] x 色[3: RGB])
print('X_Train: {0}'.format(X_train.shape))

# Y_Train: 学習用データ正解ラベル (枚数[50,000] x 分類の答え[1: 0〜9までの10分類])
print('Y_Train: {0}'.format(Y_train.shape))

# X_Train: テスト用画像データ (枚数[10,000] x 縦[32] x 横[32] x 色[3: RGB])
print('X_Test: {0}'.format(X_test.shape))

# Y_Train: テスト用データ正解ラベル (枚数[10,000] x 分類の答え[1: 0〜9までの10分類])
print('Y_Test: {0}'.format(Y_test.shape))

# 画像データ形式の確認
print('X_Train: {0}'.format(X_train[0]))

# 正解ラベルデータ形式の確認
print('Y_Train: {0}'.format(Y_train[:10]))

### 画像データの表示
一部を画像として出力する

In [ ]:
# 32 x 32 x 3 のデータ
for i in range(5):
    plt.imshow(X_train[i])
    plt.show()

### 画像データのRGB値(0〜255)を0〜1の範囲に収まるように正規化する

In [ ]:
# 画像データ(学習用、テスト用どちらも) 特徴量の正規化
X_train = X_train/255.0
X_test = X_test/255.0

X_train[0]

### 正解ラベルデータの形式を変換
例)<br>
1　→ [0, 1, 0, 0, 0, 0, 0, 0, 0, 0]<br>
5　→ [0, 0, 0, 0, 0, 1, 0, 0, 0, 0]

In [ ]:
# クラスラベルの1-hotベクトル化
Y_train = to_categorical(Y_train, 10)
Y_test = to_categorical(Y_test, 10)

Y_train[:10]

In [ ]:
# CNNの構築

# Epochs
epochs = 10

# Sequential = 直列構造のモデルを構築、addで層を追加していく。
model = Sequential()

# 畳み込み層を追加(32種類の3x3フィルタを作成・適用、same=ゼロパディング法を適用。(valid=次元削減の畳み込み))
model.add(Conv2D(32, (3, 3), padding='same',input_shape=X_train.shape[1:]))
# relu関数を適用し、負の値は0に変換
model.add(Activation('relu'))

# 畳み込み層を追加(32種類の3x3フィルタを作成・適用、valid=次元削減の畳み込みを適用。)
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))

# 2x2フィルタでMaxPooling法を適用
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

# 畳み込み層を追加(64種類の3x3フィルタを作成・適用、same=ゼロパディング法を適用。(valid=次元削減の畳み込み))
model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))

# 畳み込み層を追加(64種類の3x3フィルタを作成・適用、valid=次元削減の畳み込みを適用。)
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))

# 畳み込み層を追加(64種類の3x3フィルタを作成・適用、same=ゼロパディング法を適用。(valid=次元削減の畳み込み))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
 
# 1次元のデータに変換
model.add(Flatten())

# 隠れ層(512)を追加
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))

# 出力層を追加
model.add(Dense(10))
model.add(Activation('softmax'))
 
# 予測モデルをコンパイル(損失関数=交差エントロピー誤差、オプティマイザ=SGD、評価関数=accuracy)
# オプティマイザ：学習による重みの更新
model.compile(loss='categorical_crossentropy',optimizer='SGD',metrics=['accuracy'])
 
#訓練(epochs=世代数)
history = model.fit(X_train, Y_train, epochs=epochs)

### 予測モデルの評価を結果出力する

In [ ]:
#評価 & 評価結果出力
print(model.evaluate(X_test, Y_test))

### 今回作成したCNNモデルを可視化する

In [ ]:
#作成した予測モデルの図示化
plot_model(model, to_file='model.png',show_shapes=True)

### 世代毎の学習の過程を表示

In [ ]:
plt.figure(figsize=(10, 8))
plt.plot(range(1, epochs+1), history.history['accuracy'], label="accuracy")
plt.plot(range(1, epochs+1), history.history['loss'], label="loss")
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()